In [114]:
import plotly.graph_objects as go
import pandas as pd
import math

In [24]:
def import_covid19_data():
    '''Import covid19 data from 2019 Novel Coronavirus COVID-19 (2019-nCoV) 
    Data Repository by Johns Hopkins CSSE and output confirmed cases, deaths,
    and recovered cases into three pandas dataframes.
    '''
    
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master'\
    +'/csse_covid_19_data/csse_covid_19_time_series/'
    conf = 'time_series_19-covid-Confirmed.csv'
    death = 'time_series_19-covid-Deaths.csv'
    recov = 'time_series_19-covid-Recovered.csv'
    
    confirmed = pd.read_csv(url + conf)
    deaths = pd.read_csv(url + death)
    recov = pd.read_csv(url + recov)
    
    return confirmed, deaths, recov

In [25]:
confirmed, deaths, recovered = import_covid19_data()

In [46]:
confirmed['Province/State'].fillna(confirmed['Country/Region'], inplace=True)
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,Thailand,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,47,48,50,50,50,53,59,70,75,82
1,Japan,Japan,36.0000,138.0000,2,1,2,2,4,4,...,360,420,461,502,511,581,639,639,701,773
2,Singapore,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,117,130,138,150,150,160,178,178,200,212
3,Nepal,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Malaysia,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,50,83,93,99,117,129,149,149,197,238


In [103]:
df = pd.DataFrame([confirmed['Province/State'], 
                   confirmed.iloc[:,-1], 
                   confirmed['Lat'], 
                   confirmed['Long']]).T
df.rename(columns={df.columns[0]: "name",
                   df.columns[1]: "pop",
                   df.columns[2]: "lat",
                   df.columns[3]: "lon"}, inplace = True)

df.sort_values(by=['pop'], ascending=False, inplace=True)
df = df[df['pop']!=0]
df = df.astype({'name': 'string',
           'pop': 'int32',
           'lat': 'float32',
           'lon': 'float32'})
df.dtypes

name     string
pop       int32
lat     float32
lon     float32
dtype: object

In [126]:
df['text'] = df['name'] + '<br>Confirmed ' + (df['pop']).astype(str)
limits = [(0,10),(11,50),(51,100),(101,150),(151,len(df)-1)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []
scale = [100, 10, 5, 1, 1]

In [128]:
fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale[i],
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = 'Covid-19',
        showlegend = True,
        geo = dict(
            scope = 'world',
            landcolor = 'rgb(217, 217, 217)',
            showcountries=True,
        )
    )

fig.show()

In [129]:
fig.data

(Scattergeo({
     'lat': array([30.9756, 43.    , 32.    , 36.    , 40.    , 51.    , 46.2276, 46.8182,
                   23.3417, 33.882 ], dtype=float32),
     'lon': array([112.2707,  12.    ,  53.    , 128.    ,  -4.    ,   9.    ,   2.2137,
                     8.2275, 113.4244, 113.614 ], dtype=float32),
     'marker': {'color': 'royalblue',
                'line': {'color': 'rgb(40,40,40)', 'width': 0.5},
                'size': array([677.9 , 211.57, 127.29,  80.86,  63.91,  45.85,  44.69,  13.59,  13.56,
                                12.73]),
                'sizemode': 'area'},
     'name': '0 - 10',
     'text': array(['Hubei<br>Confirmed 67790', 'Italy<br>Confirmed 21157',
                    'Iran<br>Confirmed 12729', 'Korea, South<br>Confirmed 8086',
                    'Spain<br>Confirmed 6391', 'Germany<br>Confirmed 4585',
                    'France<br>Confirmed 4469', 'Switzerland<br>Confirmed 1359',
                    'Guangdong<br>Confirmed 1356', 'Henan<br>Con